In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import random
from open_clip import create_model_and_transforms, get_mean_std, HFTokenizer
from PIL import Image
import torch
import numpy as np
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)
# Define main parameters
model_name = 'ViT-B-16-quickgelu' # available pretrained weights ['ViT-L-14-336-quickgelu', 'ViT-B-16-quickgelu']
pretrained_weights = "../../checkpoints/unimed_clip_vit_b16.pt" # Path to pretrained weights
text_encoder_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext" # available pretrained weights ["microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract", "microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract"]
mean, std = get_mean_std()
device='cpu'
# Load pretrained model with transforms
model, _, preprocess = create_model_and_transforms(
    model_name,
    pretrained_weights,
    precision='amp',
    device=device,
    force_quick_gelu=True,
    mean=mean, std=std,
    inmem=True,
    text_encoder_name=text_encoder_name,)

tokenizer = HFTokenizer(
    text_encoder_name,
    context_length=256,
    **{},)

data_dir = '../../../../datasets/xray/'

in_class = ['normal', 'pneumonia']
in_dir = ['normal/', 'pneumonia/']

ood_dir = ['covid']
in_images = []
ood_images = []

per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:300]])
    # ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
# Prepare text prompts using different class names

text = tokenizer([x for x in in_class]).to(device)

# text = tokenizer(["%s" %x for x in in_class]).to(device)
# Load and preprocess images

# Inference
in_probs = []
ind_score = []
all_preds = []
with torch.no_grad():
    for in_image in in_images:
        image = preprocess(Image.open(in_image)).unsqueeze(0).to(device)
        text_features = model.encode_text(text)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        image_features = model.encode_image(image)
        
        probs = (image_features @ text_features.t()).detach().softmax(dim=-1).cpu().numpy()
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
        logits = (image_features @ text_features.t()).detach().cpu().numpy()
        ind_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 
        all_preds.append(preds)
    

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)

ood_score = []
with torch.no_grad():
    for in_image in ood_images:
        image = preprocess(Image.open(in_image)).unsqueeze(0).to(device)
        text_features = model.encode_text(text)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        image_features = model.encode_image(image)
        logits = (image_features @ text_features.t()).detach().cpu().numpy()
        probs = (image_features @ text_features.t()).detach().softmax(dim=-1).cpu().numpy()
        ood_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 

from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")

preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
roc_auc_score(all_gts, preds_probs)

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import random
from open_clip import create_model_and_transforms, get_mean_std, HFTokenizer
from PIL import Image
import torch
import numpy as np
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)
# Define main parameters
model_name = 'ViT-B-16-quickgelu' # available pretrained weights ['ViT-L-14-336-quickgelu', 'ViT-B-16-quickgelu']
pretrained_weights = "../../checkpoints/unimed_clip_vit_b16.pt" # Path to pretrained weights
text_encoder_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext" # available pretrained weights ["microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract", "microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract"]
mean, std = get_mean_std()
device='cpu'
# Load pretrained model with transforms
model, _, preprocess = create_model_and_transforms(
    model_name,
    pretrained_weights,
    precision='amp',
    device=device,
    force_quick_gelu=True,
    mean=mean, std=std,
    inmem=True,
    text_encoder_name=text_encoder_name,)

tokenizer = HFTokenizer(
    text_encoder_name,
    context_length=256,
    **{},)

data_dir = '../../../../datasets/xray/'

in_class = ['normal', 'pneumonia']
in_dir = ['normal/', 'pneumonia/']

ood_dir = ['imagenet/']
in_images = []
ood_images = []

per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:300]])
    # ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
# Prepare text prompts using different class names

text = tokenizer([x for x in in_class]).to(device)

# text = tokenizer(["%s" %x for x in in_class]).to(device)
# Load and preprocess images

# Inference
in_probs = []
ind_score = []
all_preds = []
with torch.no_grad():
    for in_image in in_images:
        image = preprocess(Image.open(in_image)).unsqueeze(0).to(device)
        text_features = model.encode_text(text)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        image_features = model.encode_image(image)
        probs = (image_features @ text_features.t()).detach().softmax(dim=-1).cpu().numpy()
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
        logits = (image_features @ text_features.t()).detach().cpu().numpy()
        ind_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 
        all_preds.append(preds)
    

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)

ood_score = []
with torch.no_grad():
    for in_image in ood_images:
        image = preprocess(Image.open(in_image)).unsqueeze(0).to(device)
        text_features = model.encode_text(text)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        image_features = model.encode_image(image)
        probs = (image_features @ text_features.t()).detach().softmax(dim=-1).cpu().numpy()
        logits = (image_features @ text_features.t()).detach().cpu().numpy()
        ood_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 

from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")

preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
roc_auc_score(all_gts, preds_probs)

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import random
from open_clip import create_model_and_transforms, get_mean_std, HFTokenizer
from PIL import Image
import torch
import numpy as np
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)
# Define main parameters
model_name = 'ViT-B-16-quickgelu' # available pretrained weights ['ViT-L-14-336-quickgelu', 'ViT-B-16-quickgelu']
pretrained_weights = "../../checkpoints/unimed_clip_vit_b16.pt" # Path to pretrained weights
text_encoder_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext" # available pretrained weights ["microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract", "microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract"]
mean, std = get_mean_std()
device='cpu'
# Load pretrained model with transforms
model, _, preprocess = create_model_and_transforms(
    model_name,
    pretrained_weights,
    precision='amp',
    device=device,
    force_quick_gelu=True,
    mean=mean, std=std,
    inmem=True,
    text_encoder_name=text_encoder_name,)

tokenizer = HFTokenizer(
    text_encoder_name,
    context_length=256,
    **{},)

data_dir = '../../../../datasets/xray/'

in_class = ['normal', 'pneumonia']
in_dir = ['normal/', 'pneumonia/']

ood_dir = ['Normal_CT', 'Pneumonia_CT']
in_images = []
ood_images = []

per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])


# Prepare text prompts using different class names

text = tokenizer([x for x in in_class]).to(device)

# text = tokenizer(["%s" %x for x in in_class]).to(device)
# Load and preprocess images

# Inference
in_probs = []
ind_score = []
all_preds = []
with torch.no_grad():
    for in_image in in_images:
        image = preprocess(Image.open(in_image)).unsqueeze(0).to(device)
        text_features = model.encode_text(text)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        image_features = model.encode_image(image)
        probs = (image_features @ text_features.t()).detach().softmax(dim=-1).cpu().numpy()
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
        logits = (image_features @ text_features.t()).detach().cpu().numpy()
        ind_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 
        all_preds.append(preds)
    

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)

ood_score = []
with torch.no_grad():
    for in_image in ood_images:
        image = preprocess(Image.open(in_image)).unsqueeze(0).to(device)
        text_features = model.encode_text(text)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        image_features = model.encode_image(image)
        probs = (image_features @ text_features.t()).detach().softmax(dim=-1).cpu().numpy()
        logits = (image_features @ text_features.t()).detach().cpu().numpy()
        ood_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 

from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)

# Print results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")

preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
roc_auc_score(all_gts, preds_probs)